In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [17]:
#Funcion que limpia los datos, quedandose con el nivel de educacion mas alto de casa id de los postulantes.
def education_cleaning(df):
    education_dic= {'Secundario': 1, 'Universitario':3,'Terciario/Técnico':2, 'Otro':0, 'Posgrado':4, 'Master': 5, 'Doctorado': 6}
    df['nombre_sort'] = df['nombre'].map(education_dic)
    df = df.sort_values(by='nombre_sort', ascending=False)
    df = df.drop_duplicates(subset='idpostulante',keep='first')
    return df


In [18]:
def datetime(date,format_string):
    return pd.to_datetime(date,format= format_string,errors='coerce')

In [19]:
#Leyendo los datos
data_postulant_education= pd.read_csv('fiuba_1_postulantes_educacion.csv')
data_postulant_education =education_cleaning(data_postulant_education)
data_postulant_education.count()

idpostulante    188752
nombre          188752
estado          188752
nombre_sort     188752
dtype: int64

In [20]:
data_postulant_sex_age= pd.read_csv('fiuba_2_postulantes_genero_y_edad.csv')
data_postulant_sex_age.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [21]:
data_postulation= pd.read_csv('fiuba_4_postulaciones.csv')
data_postulation.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [22]:
data_ads_details= pd.read_csv('fiuba_6_avisos_detalle.csv')
data_ads_details.count()

idaviso                 13534
idpais                  13534
titulo                  13534
descripcion             13534
nombre_zona             13534
ciudad                     47
mapacalle                 872
tipo_de_trabajo         13534
nivel_laboral           13534
nombre_area             13534
denominacion_empresa    13529
dtype: int64

In [23]:
#Mergeo los data frames sobre la informacion de edad y sexo y educacion del postulante del postulante con 
#la informacion del aviso en que se postulo tambien.
merge_postulants_info = data_postulant_education.merge(data_postulant_sex_age,on = 'idpostulante')
merge_postulants_ads = merge_postulants_info.merge(data_postulation,on = 'idpostulante')
merge_postulants_ads = merge_postulants_ads.merge(data_ads_details,on = 'idaviso')

In [24]:
merge_postulants_ads['se_postulo'] = 1
merge_postulants_ads.count()

idpostulante            2985090
nombre                  2985090
estado                  2985090
nombre_sort             2985090
fechanacimiento         2956687
sexo                    2985090
idaviso                 2985090
fechapostulacion        2985090
idpais                  2985090
titulo                  2985090
descripcion             2985090
nombre_zona             2985090
ciudad                    10159
mapacalle                199976
tipo_de_trabajo         2985090
nivel_laboral           2985090
nombre_area             2985090
denominacion_empresa    2984852
se_postulo              2985090
dtype: int64

In [25]:
#Ahora me genero un dataframe con la misma cantidad de (idpotulante-idaviso) pero esta vez de NO postulaciones
import random

def get_random_ids(df_postulants,df_ads):
    dic = {'idpostulante':[], 'idaviso':[]}
    for i in range (0,2985091):
        numero = random.randrange(0,188752)
        idpostulante = df_postulants['idpostulante'][numero]
        dic['idpostulante'].append(idpostulante)
        numero2 = random.randrange(0,13534)
        idaviso = df_ads['idaviso'][numero2]
        dic['idaviso'].append(idaviso)
    dataframe = pd.DataFrame(data=dic)
    return dataframe
        

In [26]:
random_postulant_ad = get_random_ids(merge_postulants_info,data_ads_details)


In [27]:
random_postulant_ad.head()

,idaviso,idpostulante
0,1112361959,ak4aW4m
1,1112342496,E61XA8
2,1112367799,5mPQxRk
3,1112279042,0zrd51a
4,1112262767,Bmaxm3a


In [28]:
#Mergeo los data frames sobre la informacion de edad y sexo y educacion del postulante del postulante con 
#la informacion del aviso en que se postulo tambien.
merge_postulants_info_NOTS = random_postulant_ad.merge(data_postulant_sex_age,on = 'idpostulante')
merge_postulants_info_NOTS = merge_postulants_info_NOTS.merge(data_postulant_education,on = 'idpostulante')
merge_postulants_ads_NOTS = merge_postulants_info_NOTS.merge(data_ads_details,on = 'idaviso')
merge_postulants_ads_NOTS.head()

,idaviso,idpostulante,fechanacimiento,sexo,nombre,estado,nombre_sort,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1112361959,ak4aW4m,1991-12-14,FEM,Secundario,Abandonado,1,1,Desarrolladores .NET C# nivel Jr. / de 6 a 18 ...,<p><strong>SOLUTIX</strong> está en búsqueda d...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Programación,SOLUTIX S.A.
1,1112361959,A3Xwdwl,1988-02-19,MASC,Universitario,En Curso,3,1,Desarrolladores .NET C# nivel Jr. / de 6 a 18 ...,<p><strong>SOLUTIX</strong> está en búsqueda d...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Programación,SOLUTIX S.A.
2,1112361959,5qWwLz,1983-03-17,MASC,Master,Graduado,5,1,Desarrolladores .NET C# nivel Jr. / de 6 a 18 ...,<p><strong>SOLUTIX</strong> está en búsqueda d...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Programación,SOLUTIX S.A.
3,1112361959,mzjPQ4E,1991-10-31,FEM,Universitario,En Curso,3,1,Desarrolladores .NET C# nivel Jr. / de 6 a 18 ...,<p><strong>SOLUTIX</strong> está en búsqueda d...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Programación,SOLUTIX S.A.
4,1112361959,JBdWMYe,1997-10-28,FEM,Universitario,En Curso,3,1,Desarrolladores .NET C# nivel Jr. / de 6 a 18 ...,<p><strong>SOLUTIX</strong> está en búsqueda d...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Programación,SOLUTIX S.A.


In [29]:
merge_postulants_ads_NOTS['se_postulo']=0

In [ ]:
#Concateno las postulaciones con las NO postulaciones

frames = [merge_postulants_ads,merge_postulants_ads_NOTS]
total = pd.concat(frames)
total.drop_duplicates(subset=['idpostulante','idaviso'], keep = 'first')

In [ ]:
total.sample(5)

In [18]:
total.count()

ciudad                    20436
denominacion_empresa    5968802
descripcion             5970181
estado                  5970181
fechanacimiento         5896135
fechapostulacion        2985090
idaviso                 5970181
idpais                  5970181
idpostulante            5970181
mapacalle                392011
nivel_laboral           5970181
nombre                  5970181
nombre_area             5970181
nombre_sort             5970181
nombre_zona             5970181
se_postulo              5970181
sexo                    5970181
tipo_de_trabajo         5970181
titulo                  5970181
dtype: int64

In [20]:
total.drop(columns = ['ciudad','mapacalle','nombre_sort','idpais','descripcion','fechapostulacion','titulo'])

,denominacion_empresa,estado,fechanacimiento,idaviso,idpostulante,nivel_laboral,nombre,nombre_area,nombre_zona,se_postulo,sexo,tipo_de_trabajo
0,Manpower,En Curso,NaN,1112145679,4rP6q15,Senior / Semi-Senior,Doctorado,Call Center,Gran Buenos Aires,1,NO_DECLARA,Full-time
1,Manpower,En Curso,1998-03-14,1112145679,owav2La,Senior / Semi-Senior,Doctorado,Call Center,Gran Buenos Aires,1,FEM,Full-time
2,Manpower,Graduado,1989-05-05,1112145679,2zQMWYa,Senior / Semi-Senior,Doctorado,Call Center,Gran Buenos Aires,1,FEM,Full-time
3,Manpower,En Curso,1996-09-06,1112145679,PmJOPpZ,Senior / Semi-Senior,Doctorado,Call Center,Gran Buenos Aires,1,FEM,Full-time
4,Manpower,Graduado,1970-03-20,1112145679,EzpRZQ0,Senior / Semi-Senior,Doctorado,Call Center,Gran Buenos Aires,1,FEM,Full-time
5,Manpower,En Curso,1981-07-30,1112145679,5mrlM6G,Senior / Semi-Senior,Doctorado,Call Center,Gran Buenos Aires,1,FEM,Full-time
6,Manpower,En Curso,1987-05-28,1112145679,A3AAoZY,Senior / Semi-Senior,Master,Call Center,Gran Buenos Aires,1,FEM,Full-time
7,Manpower,Abandonado,1970-01-20,1112145679,6kKLpx,Senior / Semi-Senior,Master,Call Center,Gran Buenos Aires,1,FEM,Full-time
8,Manpower,Graduado,1982-05-04,1112145679,3N2D3YQ,Senior / Semi-Senior,Master,Call Center,Gran Buenos Aires,1,FEM,Full-time
9,Manpower,Abandonado,1984-05-25,1112145679,5dJqez,Senior / Semi-Senior,Master,Call Center,Gran Buenos Aires,1,FEM,Full-time


In [27]:
total['edadPostulante'] = datetime(total['fechanacimiento'],'%Y-%m-%d')

0     NaN
1    20.0
2    29.0
3    22.0
4    48.0
Name: edadPostulante, dtype: float64

In [24]:
total['edadPostulante'] = total['edadPostulante'].map(lambda x: 2018 - x.year)


KeyError: 'edadPostulante'

In [ ]:
total.drop(columns = ['fechanacimiento'])

In [ ]:
# estado Mapping 
total['estadoEncoded'] = total['estado'].map( {'En Curso': 0, 'Abandonado': 2, 'Graduado': 1, '': 0} ).astype(int)
# nivelEducativo Mapping 
total['nivelEducativoEncoded'] = total['nombre'].map( {'Secundario': 1, 'Universitario':3,'Terciario/Técnico':2, 'Otro':0, 'Posgrado':4, 'Master': 5, 'Doctorado': 6}
                                       ).astype(int)
# tipoDeTrabajo Mapping 
total['tipoDeTrabajoEncoded'] = total['tipo_de_trabajo'].map( {'Full.time':0, 'Part-time':1, 'Teletrabajo':2, 'Por Horas': 3, 
                                                              'Pasantia': 4, 'Temporario': 5, 'Por Contrato': 6, 
                                                              'Fines de Semana': 7, 'Primero empleo': 8} ).astype(int)
 

    
# Nivel de trabajo Mapping
total['nivelLaboralEncoded'] = total['nivel_laboral'].map( {'Senior / Semi-Senior':0, 'Junior':1, 'Otro':2, 'Jefe / Supervisor / Responsable': 3, 
                                                              'Gerencia / Alta Gerencia / Dirección': 4} ).astype(int)


#Sexo Mapping

total['sexoEncoded'] = total['sexo'].map({'FEM':0, 'MASC':1, 'NO_DECLARA':2}).astype(int)


# Mapping edad del postulante
total.loc[ total['edadPostulante'] <= 21, 'edadEncoded']                         = 0
total.loc[(total['edadPostulante'] > 21) & (total['edadPostulante'] <= 26), 'edadEncoded'] = 1
total.loc[(total['edadPostulante'] > 26) & (total['edadPostulante'] <= 30), 'edadEncoded'] = 2
total.loc[(total['edadPostulante'] > 30) & (total['edadPostulante'] <= 40), 'edadEncoded'] = 3
total.loc[ total['edadPOstulante'] > 40, 'edadEncoded'] = 4

In [ ]:
drop_elements = ['idaviso','idpostulante','nombre','tipo_de_trabajo','estado','sexo','edadPostulante','denominacion_empresa','nivel_laboral','nombre_area','nombre_zona']
postulations_encoded = total.drop(drop_elements, axis = 1)

In [ ]:
cv = KFold(n_splits=10) # Numero deseado de "folds" que haremos
accuracies = list()
max_attributes = len(list(postulations_encoded))
depth_range = range(1, max_attributes + 1)
 
# Testearemos la profundidad de 1 a cantidad de atributos +1
for depth in depth_range:
    fold_accuracy = []
    tree_model = tree.DecisionTreeClassifier(criterion='entropy',
                                             min_samples_split=10000,
                                             min_samples_leaf=100,
                                             max_depth = depth,)
    for train_fold, valid_fold in cv.split(postulations_encoded):
        f_train = postulations_encoded.loc[train_fold] 
        f_valid = postulations_encoded.loc[valid_fold] 
 
        model = tree_model.fit(X = f_train.drop(['se_postulo'], axis=1), 
                               y = f_train["se_postulo"]) 
        valid_acc = model.score(X = f_valid.drop(['se_postulo'], axis=1), 
                                y = f_valid["se_postulo"]) # calculamos la precision con el segmento de validacion
        fold_accuracy.append(valid_acc)
 
    avg = sum(fold_accuracy)/len(fold_accuracy)
    accuracies.append(avg)
    
# Mostramos los resultados obtenidos
df = pd.DataFrame({"Max Depth": depth_range, "Average Accuracy": accuracies})
df = df[["Max Depth", "Average Accuracy"]]
print(df.to_string(index=False))

In [ ]:
# Crear arrays de entrenamiento y las etiquetas que indican si se postulo o no. 
y_train = postulations_encoded['se_postulo']
x_train = postulations_encoded.drop(['se_postulo'], axis=1).values 
 
# Crear Arbol de decision con profundidad = 4
decision_tree = tree.DecisionTreeClassifier(criterion='entropy',
                                            min_samples_split=10000,
                                            min_samples_leaf=100,
                                            max_depth = 4)#depende lo q me de arriba
decision_tree.fit(x_train, y_train)